In [22]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

import torch
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim

import torchvision
import torchvision.datasets as MNIST
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset

from PIL import Image

USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda:0" if USE_CUDA else 'cpu')
print(DEVICE)

cuda:0


In [6]:
os.getcwd()

'/home/jovyan/졸업작품/UNLV_Project'

In [2]:
train_path = "Dataset/MNIST_training.csv"
test_path = "Dataset/MNIST_test.csv"

train = pd.read_csv(train_path)
test = pd.read_csv(test_path)

print(f"train data shape is {train.shape}, test data shape is {test.shape}")
print(test.head())

train data shape is (949, 785), test data shape is (50, 785)
   label  pixel0  pixel1  pixel2  pixel3  pixel4  pixel5  pixel6  pixel7  \
0      0       0       0       0       0       0       0       0       0   
1      0       0       0       0       0       0       0       0       0   
2      0       0       0       0       0       0       0       0       0   
3      0       0       0       0       0       0       0       0       0   
4      0       0       0       0       0       0       0       0       0   

   pixel8  ...  pixel774  pixel775  pixel776  pixel777  pixel778  pixel779  \
0       0  ...         0         0         0         0         0         0   
1       0  ...         0         0         0         0         0         0   
2       0  ...         0         0         0         0         0         0   
3       0  ...         0         0         0         0         0         0   
4       0  ...         0         0         0         0         0         0   

   pixel780  

In [7]:
train_y = train[["label"]]
train_x = train.drop(columns = "label")

test_y = test[["label"]]
test_x = test.drop(columns = "label")

##pixel -> image, 각각의 열마다 이미지를 만들어주기, MNIST 이미지가 28에서 끊어주기.

##Train data
os.makedirs("Dataset/train_images", exist_ok=True)
for idx in range(len(train_x)):
    image = train_x.iloc[idx].values.reshape(28,28)
    plt.imsave(f"Dataset/train_images/image_{idx:03d}.png", image, cmap='gray')
    if idx % 100 == 0: 
        print(f"{idx} images processed.")
        
##test Data
os.makedirs("Dataset/test_images", exist_ok=True)

for idx in range(len(test_x)):
    image = test_x.iloc[idx].values.reshape(28,28)
    plt.imsave(f"Dataset/test_images/image_{idx:03d}.png", image, cmap='gray')    
    if idx % 100 == 0:  
        print(f"{idx} images processed.")

0 images processed.
100 images processed.
200 images processed.
300 images processed.
400 images processed.
500 images processed.
600 images processed.
700 images processed.
800 images processed.
900 images processed.
0 images processed.


In [23]:
# Transforms
transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),  # Convert PIL Image to Tensor
    transforms.Normalize((0.5,), (0.5,))  # Normalize: mean=0.5, std=0.5
])

In [24]:
class MyDataset(Dataset):
    def __init__(self, images_dir, labels, transform=None):
        """
        Args:
            images_dir (str): Directory containing the images.
            labels (pd.DataFrame): DataFrame containing the labels.
            transform (callable, optional): Optional transforms to be applied on an image.
        """
        self.images_dir = images_dir
        self.labels = labels
        self.transform = transform
        self.image_files = sorted(os.listdir(images_dir))  # Ensure consistent order

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        # Load image
        image_path = os.path.join(self.images_dir, f"image_{idx:03d}.png")
        image = Image.open(image_path).convert("L")  # Convert to grayscale

        # Apply transforms
        if self.transform:
            image = self.transform(image)

        # Get label
        label = self.labels.iloc[idx].values[0]  # Assuming labels are in DataFrame

        return image, label

In [25]:
# Paths
train_images_dir = "Dataset/train_images"
test_images_dir = "Dataset/test_images"

train_labels = pd.read_csv("Dataset/MNIST_training.csv")[["label"]]
test_labels = pd.read_csv("Dataset/MNIST_test.csv")[["label"]]

# Dataset and DataLoader
train_dataset = MyDataset(images_dir=train_images_dir, labels=train_labels, transform=transform)
test_dataset = MyDataset(images_dir=test_images_dir, labels=test_labels, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)


In [30]:
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU()
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(out_channels)
        )
        self.downsample = downsample
        self.relu = nn.ReLU()
        self.out_channels = out_channels

    def forward(self, x):
        residual = x

        # Apply convolution layers
        out = self.conv1(x)
        out = self.conv2(out)

        # Downsample residual if necessary
        if self.downsample:
            residual = self.downsample(x)

        # Add residual and apply activation
        out += residual
        out = self.relu(out)
        return out


In [31]:
class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes = 10):
        super(ResNet, self).__init__()
        self.inplanes = 64
        self.conv1 = nn.Sequential(nn.Conv2d(1, 64, kernel_size = 7, stride = 2, padding = 3),
                                  nn.BatchNorm2d(64),
                                  nn.ReLU())
        self.maxpool = nn.MaxPool2d(kernel_size = 3, stride = 2, padding = 1)
        self.layer0 = self._make_layer(block, 64, layers[0], stride = 1)
        self.layer1 = self._make_layer(block, 128, layers[1], stride = 2)
        self.layer2 = self._make_layer(block, 256, layers[2], stride = 2)
        self.layer3 = self._make_layer(block, 512, layers[3], stride = 2)
        self.avgpool = nn.AvgPool2d(7, stride = 1)
        self.fc = nn.Linear(512, num_classes)
        
    def _make_layer(self, block, planes, blocks, stride=1):
        downsample = None

        # Downsample if stride > 1 or channels don't match
        if stride != 1 or self.inplanes != planes:
            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes

        for _ in range(1, blocks):
            layers.append(block(self.inplanes, planes))

        return nn.Sequential(*layers)

    
    def forward(self, x):
        x = self.conv1(x)
        x = self.maxpool(x)
        x = self.layer0(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        
        x = self.avgpool(x)
        x = x.view(x.size(0),-1)
        x = self.fc(x)
        
        return x

In [32]:
num_classes = 10
epochs = 20
batch_size = 16
learning_rate = 0.01

model = ResNet(ResidualBlock, [2, 2, 2, 2], num_classes=10).to(DEVICE)

##Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate, weight_decay = 0.001, momentum = 0.9)

total_step = len(train_loader)

In [35]:
for epoch in range(epochs):
    for i,(images, labels) in enumerate(train_loader):
        images = images.to(DEVICE)
        labels = labels.to(DEVICE)
        
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print ('Epoch [{}/{}], Loss: {:.4f}' .format(epoch+1, epochs, loss.item()))
    
    ##test
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in test_loader:
            images = images.to(DEVICE)
            labels = labels.to(DEVICE)
            outputs = model(images)
            
            _,predicted = torch.max(outputs.data,1)
            total+=labels.size(0)
            correct += (predicted==labels).sum().item()
        print("Accuracy of the network on the {} test images {} %".format(50, 100*correct/total))


Epoch [1/20], Loss: 0.0033
Accuracy of the network on the 50 test images 94.0 %
Epoch [2/20], Loss: 0.0006
Accuracy of the network on the 50 test images 94.0 %
Epoch [3/20], Loss: 0.0217
Accuracy of the network on the 50 test images 94.0 %
Epoch [4/20], Loss: 0.1177
Accuracy of the network on the 50 test images 92.0 %
Epoch [5/20], Loss: 0.0003
Accuracy of the network on the 50 test images 96.0 %
Epoch [6/20], Loss: 0.0055
Accuracy of the network on the 50 test images 98.0 %
Epoch [7/20], Loss: 0.0038
Accuracy of the network on the 50 test images 94.0 %
Epoch [8/20], Loss: 0.0010
Accuracy of the network on the 50 test images 96.0 %
Epoch [9/20], Loss: 0.0002
Accuracy of the network on the 50 test images 98.0 %
Epoch [10/20], Loss: 0.0065
Accuracy of the network on the 50 test images 96.0 %
Epoch [11/20], Loss: 0.0007
Accuracy of the network on the 50 test images 98.0 %
Epoch [12/20], Loss: 0.0113
Accuracy of the network on the 50 test images 94.0 %
Epoch [13/20], Loss: 0.0008
Accuracy 